In [4]:
import json
import random

from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Any
import cv2
import numpy as np
from PIL import Image

BASE_DIR = Path.cwd().resolve().parent

@dataclass
class Paths:
    # Using resolve() helps ensure paths are absolute and correct
    TRAIN_DIR: Path = BASE_DIR / "data" / "train"
    TRAIN_VIDEO: Path = TRAIN_DIR / "videos"
    TRAIN_JSON: Path = TRAIN_DIR / "train.json"

paths = Paths()

class VideoTrainData:
    def __init__(self, train_json_path: Path, video_dir: Path):
        self.train_json_path = train_json_path
        self.video_dir = video_dir
        self.data: List[Dict[str, Any]] = self._load_data()
        self._format_choices()

    def _load_data(self) -> List[Dict[str, Any]]:
        """Load and parse the training JSON file."""
        if not self.train_json_path.exists():
            raise FileNotFoundError(f"Train JSON file not found: {self.train_json_path}")

        with open(self.train_json_path, 'r', encoding='utf-8') as f:
            content = json.load(f)
            # Ensure we return a list, even if 'data' key is missing
            return content.get("data", [])
        
    def load_video_frame(self, video_path: Path, frame_idx: int) -> Image.Image:
        """Loads a specific frame from a video file using OpenCV."""
        # Convert Path to string for OpenCV
        cap = cv2.VideoCapture(str(video_path))
        
        if not cap.isOpened():
            raise FileNotFoundError(f"Could not open video file: {video_path}")

        # Set the position of the video to the specific frame index
        # method: cv2.CAP_PROP_POS_FRAMES = 1
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        
        ret, frame = cap.read()
        cap.release()

        if not ret:
            # Fallback or Error: Frame likely out of bounds
            raise ValueError(f"Could not read frame {frame_idx} from {video_path}")

        # OpenCV loads in BGR, convert to RGB for PIL/PyTorch
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        return Image.fromarray(frame_rgb)
    
    def _format_choices(self):
        """Formats the multiple choice options into a single string."""
        for entry in self.data:
            choices = entry.get('choices', [])

            if not choices and 'question' in entry:
                 choices = entry['question'].get('choices', [])
                 
            formatted_choices = ("\n").join(
                choice for choice in choices
            )
            entry['formatted_choices'] = formatted_choices
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx):
        """Returns a dictionary containing the processed image and metadata."""
        item = self.data[idx]
        
        # Identify Video Path
        video_filename = item.get('video_path')
        if not video_filename:
            raise KeyError(f"Entry at index {idx} is missing 'video_path'")

        video_path = self.video_dir / video_filename

        # 2. Identify Frame Index
        # Assuming JSON has 'frame_idx'. If not, defaults to frame 0.
        # For RoadBuddy, often you need a specific timestamp or frame.
        frame_idx = item.get('frame_idx', 0) 

        # 3. Load Image
        try:
            image = self.load_video_frame(video_path, frame_idx)
        except Exception as e:
            print(f"Error loading video {video_filename}: {e}")
            # Return a blank image or handle error appropriately for training
            image = Image.new('RGB', (224, 224), color='black')

        # 4. Return combined data
        return {
            "image": image,
            "question": item.get('question', ''),
            "choices": item.get('formatted_choices', ''),
            "answer": item.get('answer', ''), # If training labels exist
            "original_data": item # Keep raw data just in case
        }

    def get_random_sample(self):
        if not self.data:
            return None
        return random.choice(self.data)

ModuleNotFoundError: No module named 'cv2'